In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as nnf



class resenet_block(nn.Module):
    def __init__(self,channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels,channels,3,padding='same')
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels,channels,3,padding='same')
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self,x):
        y = self.conv1(x)
        y = nnf.relu(self.bn1(y))
        y = self.conv2(y)
        y = nnf.relu(self.bn2(x+y))
        return y
        



    

In [21]:
sec = []
for i in range(10):
    sec.append(resenet_block(12))

x = torch.ones(8,12,10,10)    

for comp in sec:
    x = comp(x)

x.shape

    

torch.Size([8, 12, 10, 10])